In [102]:
import torch.nn as nn
import torch.nn.functional as F
torch.cuda.set_device(0)
import numpy as np
import pandas as pd
import urllib.request
from pathlib import Path

### Load Data

In [10]:
PATH = Path('/mnt/data/extracts/rnns/')
txt = (PATH/'nietzsche.txt').read_text()

### Create Vocab

In [25]:
chars = set(txt)
padding_char = "\0"
chars.add(padding_char)
chars = sorted(list(chars))
c2i = {c:i for i, c in enumerate(chars)}
i2c = {i:c for i, c in enumerate(chars)}

### Convert Text

In [26]:
inp = [c2i[c] for c in txt]

In [89]:
x1 = [inp[i] for i in range(0, len(inp) - 3, 3)]
x2 = [inp[i] for i in range(1, len(inp) - 3, 3)]
x3 = [inp[i] for i in range(2, len(inp) - 2, 3)]
x4 = [inp[i] for i in range(3, len(inp) - 1, 3)]

In [90]:
x = np.concatenate([np.vstack(x1), np.vstack(x2), np.vstack(x3)], axis=1)
y = np.vstack(x4)

### Build Model

In [ ]:
class char_rnn(nn.Module):
    def __init__(self, vocab_size, n_fac, n_hidden):
        self.emb = nn.Embedding(vocab_size, n_fac)
        self.hidden = nn.Linear(n_fac, n_fac)
        self.